In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

# 1. train.csv

In [ ]:
train = pd.read_csv('data/train.csv')

## 1-1. id
- NULL 없음
- TRAIN_000부터 TRAIN_643까지 644개의 고유한 값 -> data indexing을 위한 것일 뿐

In [ ]:
print(len(train['id']))
print(train['id'].nunique())

- 질문_1, 질문_2 : 샘플 별 동일한 내용으로 구성된 질문 2개

## 1-2. 질문_1
- NULL 없음
- '장판의 장점이 뭐야?'는 2개, 나머지는 1개 -> 643개의 unique 질문

## 1-3. 질문_2
- NULL 없음
- '오리지널징크의 장점은 무엇인가요?'는 2개, 나머지는 1개 -> 643개의 unique 질문
---
- 하지만 질문_1과 질문_2이 동일한 내용이라고 보기는 어려움.
- 존댓말 + 조금 더 구체적인 질문이긴 하나, '장판의 단점이 뭐야?' -> '장판의 장점과 단점에는 무엇이 있나요?' 이렇게 되는 경우가 있음.
    - 하지만 이 경우에 5개의 답변은 모두 '단점'만 설명하고 있음. 질문_1로 5개 답변을 생성하고, 질문_2는 질문_1의 유사 답변으로 자동 생성하게 한 것 같다는 생각이 든다. 사실상 질문_1이 제대로 된 질문일 수도 있을 것 같다.
    - 질문1 -> GPT로 답변 5개 생성 / 질문_1 -> GPT로 비슷한 질문 1개 생성... 이런 느낌인 것 같음.
- 다른 예시, 이때에도 5개 답변이 색상을 얘기하는 게 아니라 요소를 얘기함.
    - 질문1: '인테리어에서 유럽의 전통적인 스타일을 살리려면 어떤 요소를 고려해야 하나요?'
    - 질문2: '유럽의 전통적인 스타일을 살리기 위해 어떤 색상을 활용해야 할까요?'
---
- 안 물어본 것도 답변하는 경우가 있음
    - 벽에 뚫린 구멍이 이상해 -> 벽이나 천장에 잘못 뚫린 구멍을 "타공하자"라고 합니다. **_타공하자가 발생하는 원인과 그에 대한 책임 소재, 그리고 해결방법에 대해 설명드리겠습니다._** 1. 부실 시공 - 원인: 적절하지 않은 도구로 작업하거나 타공 부위의 크기를 정확히 측정하지 않고 작업하면 타공불량이 발생할 수 있습니다. - 책임 소재: 시공자 - 해결 방법: 타공 부위의 크기를 정확히 측정하고 올바른 도구를 사용하여 재작업해야 합니다.
- pattern = r'[^a-zA-Z가-힣()\d.,?!\s\-`]'에서 모두 커버가 됨.
    - 도막방수공사가 뭐야?\` -> 맨 뒤에 `가 붙어 있음.

In [ ]:
display(train[train['질문_1'] == '장판의 장점이 뭐야?'])
print(len(train['질문_1'].value_counts()))

display(train[train['질문_2'] == '오리지널징크의 장점은 무엇인가요?'])
print(len(train['질문_2'].value_counts()))

In [ ]:
display(train[train['id'] == 'TRAIN_313'])

In [ ]:
display(train[train['질문_1'].str.contains('인테리어에서 유럽의 전통적인 스타일을 살리려면 어떤 요소를 고려해야 하나요?')])

In [ ]:
pattern = r'[^a-zA-Z가-힣()\d.,?!\s\-]'
mask = train[['질문_1', '질문_2']].apply(lambda x: x.str.contains(pattern)).any(axis=1)
print(len(train[mask]))
train[mask]

# category
- NULL 없음
- '마감재', '인테리어', '시공', '마감하자', '건축구조', '기타', '타 마감하자'
- 질문의 카테고리니까, 644 * 2개 만큼의 데이터가 존재하는 셈
    - 질문을 보고 카테고리를 우선 분류하는 모델을 사용해보면 어떨까? -> 이후 RAG 사용하여 해당 카테고리만 읽으면 조금 더 정확해질 수도?
    - 어차피 그런 분류 모델이라면 답변도 데이터로 사용해볼 수 있겠다.

In [ ]:
train['category'].value_counts()

# 답변_N
- NULL 없음
- 답변1 -> 답변2 -> ... 순서대로 답변 내용이 추가되는 느낌. 비슷하거나 동일한 말이 반복되는 경우가 있음
- 답변의 길이는 답변_1이 평균 95개 문자, 답변_2부터 답변_5까지는 195개 문자. 답변_1이 눈에 띄게 짧고, 나머지는 거의 비슷한 길이를 가짐.
- 답변에 오타가 존재할 수 있음
    - 노출콘크리트는 모던하며 다양한 인테리어 효ㅕ과와 공간의 확장감을 주고 별도의 마감재가 필요없으며 표면의 강도가 높다는 장점이 있습니다.
- 확실히 GPT로 답변 생성했다는 것이 느껴지는 부분
    - \## 수정된 답변 발포우레탄폼은 폴리우레탄 수지를 발포시켜 고화된 스폰지 모양의 외장재를 의미합니다. 주로 건물의 단열재나 수납 공간 등을 채우는 데 사용되며, 경량이면서도 견고한 재질로 인해 다양한 용도로 활용됩니다. 특히, 발포우레탄폼은 부식이나 노후 등에도 강한 내구성을 가지고 있어 건축 및 인테리어에서 효율적으로 활용됩니다.
- pattern = r'[^a-zA-Z가-힣\d.,?!\s()-~"\'·:%&#*℃㎥°+×÷‘’]' 이걸로 3개 제외 모두 커버할 수 있음
    - 깨진 글자가 존재하지는 않으나 ℃ <-> °C 처럼 같은 의미인데 다르게 표현되는 경우도 있음.
    - 질문에서는 특수문자가 ().,?!- 말고는 존재하지 않음. -> 테스트 데이터에서도 그냥 없애버리는 것도 방법일듯.

In [ ]:
train[['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']].head(3)

In [ ]:
print(train['답변_1'].apply(len).mean())
print(train['답변_2'].apply(len).mean())
print(train['답변_3'].apply(len).mean())
print(train['답변_4'].apply(len).mean())
print(train['답변_5'].apply(len).mean())

In [ ]:
pattern = r'[^a-zA-Z가-힣\d.,?!\s()-~"\'·:%&#*℃㎥°+×÷‘’]'
# ℃ 2개
# ㎥ 1개
# ㅕ 1개
# ° 1개
# × ÷ 1개
# ‘’ 1개
# 
mask = train[['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']].apply(lambda x: x.str.contains(pattern)).any(axis=1)
print(len(train[mask]))
train[mask]
